# Customer Analysis Project

## User permissions

In [ ]:
ls -l

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%pylab inline

import sklearn as sk
import sklearn.tree as tree
from IPython.display import Image  
import pydotplus

In [ ]:
#print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#display all the columns
pd.options.display.max_columns = None

## Data Preprocessing

In [ ]:
df  = pd.read_csv('marketing_campaign.csv', sep='\t')

## Data Analyzation

Analyze dataset to see what needs to be dropped, binarized, and what columns have null values

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
for c in df.columns:
    print(c + ' ' + str(df[c].nunique()) )

In [ ]:
#make a copy to clean up the data in the way we want while also keeping the original data set
df2 = df.copy()

### Data Cleaning

In [ ]:
#drop the income rows that are null, as there are only 24, which represents 1% of the data
df2.dropna(how = 'any', subset =['Income'], inplace = True)

In [ ]:
#drop attributes that do not provide value to the analysis
df2.drop(['ID','Dt_Customer','Z_CostContact','Z_Revenue','Complain'], axis=1,  inplace = True)

In [ ]:
df2.columns

In [ ]:
df2.groupby('Marital_Status')['Marital_Status'].size()

In [ ]:
#Yolo, Absurd, and Alone are a small section of  the marital status column.
#replace those values to Single. All categories signify Single
df2.Marital_Status.replace(['YOLO','Absurd','Alone'], value = 'Single', inplace = True)

In [ ]:
df2.Marital_Status.unique()

In [ ]:
#2n Cycle is another name for master's, so replace 2n cycle with master
df2.Education.replace(to_replace ='2n Cycle', value = 'Master', inplace = True)

In [ ]:
df2.Education.unique()

In [ ]:
#make the categorial data into numerical data
df2 = pd.get_dummies(df2,columns=['Marital_Status','Education'])

In [ ]:
df2.head()

Add columns that we will use in our findings and reorder the columns based off what we thinks makes sense. Use different ways to add the new columns and redorder the columns, depending on whihc way is the easiest to do

In [ ]:
df2['TotalChildren'] = (df2.Kidhome + df2.Teenhome)

In [ ]:
df2['TotalCmpAccpt'] = (df2.AcceptedCmp3+df2.AcceptedCmp4+df2.AcceptedCmp5+df2.AcceptedCmp1+\
                        df2.AcceptedCmp2+df2.Response)

In [ ]:
#rename the attributes so they are more coordinated, shorter, and readable
df2.columns =['BirthYear',
            'Income',
            'Kids',
            'Teens',
            'Recency',
            'Wine',
            'Fruits',
            'Meat',
            'Fish',
            'Sweets',
            'Gold',
            'DealPurchases',
            'WebPurchases',
            'CatalogPurchases',
            'StorePurchases',
            'MonthWebVisits',
            'Campaign3',
            'Campaign4',
            'Campaign5',
            'Campaign1',
            'Campaign2',
            'Campaign6', 
            'Divorced',
            'Married',
            'Single',
            'Together',
            'Widow', 
            'Basic', 
            'Graduation', 
            'Master\'s', 
            'Phd',
            'TotalChildren',
            'TotalCmpAccpt'] 

In [ ]:
list_col = df2.columns.tolist()

In [ ]:
list_col

In [ ]:
#sort the campaigns so it is easier to read
list_col[list_col.index('Campaign3'):list_col.index('Campaign6')]\
                = sorted(list_col[list_col.index('Campaign3'):list_col.index('Campaign6')])

In [ ]:
#order the added columns
list_col.insert(4, list_col.pop(-2))
list_col.insert(df2.columns.get_loc('Campaign6')+2, list_col.pop(-1))

In [ ]:
#assign the renamed and redorded columns to the clean dataframe
df2 = df2.reindex(columns= list_col)

In [ ]:
df2.head()

## Data set description

After cleaning, we have a DataFrame df2, which reports the enrolled customers of a certain company (one customer per row ). The columns are:

* **BirthYear**: Customer's year of birth
* **Income**: Customer's annual household income
* **Kids**: Number of kids in each customer's household
* **Teens**: Number of teens in each customer's household
* **TotalChildren**: Number of kids and teens in each customer's household
* **Recency**: Number of days since customer's last purchase
* **Wine, Fruits, Meat, Fish, Sweets, and Gold** : The amount the customer spent on each product in the last 2 years
* **DealPurchases**: The number of purchases customer made with a discount
* **WebPurchases**: The number of purchases customer made through the company's website
* **CatalogPurchases**: The number of purchases customer made using a catalog
* **StorePurchases**: The number of purchases customer made in the store
* **MonthWebVisits**: The number of times a customer visited the company's website in the last month
* **Campaign 1-6**: Binary attibute that indicates whether the customer accepted the offer in each campaign
* **TotalCmpAccpt**: The maximum number of accepted campaigns per customer
* **Divorced, Married, Single, Together, Widow**: Binary attibute that indicates the customer's marital status
* **Basic, Graduation, Master's, Phd**: Binary attibute that indicates the customer's highest education level 

# Interesting Finding 1:  Sweets and Children Correlation

### Is there a correlation between children and the amount of purchases of certain products?

In [ ]:
corr = df2.corr()

In [ ]:
corr.loc['TotalChildren'].nsmallest()

#### What is the count of customers with total number of children?

In [ ]:
df2.groupby('TotalChildren')['TotalChildren'].size()

In [ ]:
#percent of customers that have at least one child
(df2.TotalChildren !=0).sum()/len(df2)

## Summary of Finding:
Customers who have more children tend to buy less sweet products. 

## Graph below shows a decrease in sweets purchases as children number go up

In [ ]:
sns.catplot(x = 'TotalChildren',y='Sweets',kind ='bar',data=df2, aspect = 2)

## Managerial Decision
71.44% of customers have at least one child. 
To encourage these customers to buy more of sweets products, a discount on sweets can be given to these customers. 
At the same time, offering more child-oriented sweets may also increase the sale of sweets to this demographic.


# Interesting Finding 2 : Monthly Web Visits, Web Store Purchases, and Income Relation

### Lets Identify customers through Clustering

In [ ]:
from sklearn.cluster import KMeans
clu = KMeans(n_clusters=2,random_state=0)
#after running the cross validation code, KMeans = 2 had one of the highest siloutte scores
#this showed the most interesting cluster

#### Using KMeans 

In [ ]:
#through discovery of these columns of being highly correlated, we removed them for clustering
df_clust = df2.drop(columns =['Kids', 'Teens', 'Wine', 'Fruits', 'Meat', 'Fish', 'Sweets',\
                         'Gold','DealPurchases','CatalogPurchases', 'StorePurchases',\
                         'Campaign1', 'Campaign2', 'Campaign3','Campaign4', 'Campaign5','Campaign6'], axis =1)

In [ ]:
clu.fit(df_clust)
df_clust['Clustered'] = clu.labels_

In [ ]:
df_clust.groupby('Clustered').mean()

In [ ]:
df_clust.groupby('Clustered')[['Income','WebPurchases','MonthWebVisits']].mean()

## Summary of Finding:

Lower income customers visit the web more every month, while customers with a higher income make more online purchases.

In [ ]:
#bin values chosen based on clusters, discovery showed interesting results
df_clust['binned_income'] = pd.cut(df_clust.Income,bins=[0,34000,71000,df_clust.Income.max()])

In [ ]:
df_clust.groupby('binned_income')[['WebPurchases','MonthWebVisits']].mean().plot()

In [ ]:
df_clust.pivot_table(index = 'binned_income',\
                     values=['WebPurchases','MonthWebVisits'])

In [ ]:
df_clust.pivot_table(index = 'binned_income',\
                     values=['WebPurchases','MonthWebVisits']).plot()

## Monthly Web Visits is more in case of lower income people showing a downward trend

In [ ]:
sns.catplot(y='MonthWebVisits',x ='binned_income',data=df_clust,kind='bar',aspect=3)

## Graph below shows a gradual increase in web purchases with increasing income

In [ ]:
sns.catplot(y='WebPurchases',x ='binned_income',data=df_clust,kind='bar',aspect=3)

## Managerial Decision
In order to increase the purchase rate of lower income customers, they should be targeted with online discounts after they have visited the website 3 times in one month. 

# Interesting Finding 3 : Classify who has accepted a previous campaign

### How many customers have accepted at least one offer from a campaign?

In [ ]:
#create attribute to track who has accepted a previous campaign
df2['AccptPrevCmp'] = (df2.TotalCmpAccpt > 0)*1.0
#percent of customers who have accepted a previous campaign
(df2.AccptPrevCmp.sum())/len(df2)

In [ ]:
#show new variable column
df2.head()

## Can we classify the customers who will accept at least one campaign offer?

### Use a decision tree to classify

In [ ]:
#create X by dropping columns associated with the attribute AccptPrevCmp
X = df2.drop(columns=['Campaign1','Campaign2','Campaign3','Campaign4',\
                      'Campaign5','Campaign6','AccptPrevCmp', 'TotalCmpAccpt'])
#create the Y column we're trying to predict and explain
Y = df2.AccptPrevCmp
#create and fit decision tree
dt = tree.DecisionTreeClassifier(max_depth=2) 
dt.fit(X,Y)

## Summary of Finding:
Customers who have accepted at least one campaign tend to make more than \\$82,250 and spend more than \\$666.5 in wine.

In [ ]:
#wine was partially binned based on the decision tree.
#the other binned levels were based on discovery.
df2['binned_Wine'] = pd.cut(df2.Wine,bins=[0,500,650,800,1500], include_lowest=True)

# Income was binned based on the decision tree.
df2['binned_Income'] = pd.cut(df2.Income,bins=[0,82250,666666],include_lowest=True)

## Graph shows a gradual increase in accepting previous campaigns as income and amount of wine bought increases

In [ ]:
sns.catplot(data=df2, x='binned_Wine',y='AccptPrevCmp',hue='binned_Income',kind='point',aspect=2)

In [ ]:
df2.drop(columns=['AccptPrevCmp','binned_Wine','binned_Income'], axis =1 ,inplace = True)

## Managerial Decision
In order to reduce campaign marketing costs and increase campaign participation rate, the next campaign should be targeted at those who have income over \\$82,250 and have purchased at least \\$500 in wine.